In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np


# purpose를 표준 카테고리로 매핑하는 함수

def map_purpose(title):
    """자유 텍스트를 표준 카테고리로 변환합"""
    if not isinstance(title, str):
        return 'other'
    
    title = title.lower() # 소문자로 변환하여 일관성 유지

    if 'debt' in title or 'consolidation' in title:
        return 'debt_consolidation'
    if 'credit' in title or 'card' in title:
        return 'credit_card'
    if 'home' in title or 'house' in title or 'mortgage' in title:
        return 'home_improvement'
    if 'car' in title or 'auto' in title:
        return 'car'
    if 'business' in title:
        return 'small_business'
    if 'major' in title or 'purchase' in title:
        return 'major_purchase'
    if 'medical' in title:
        return 'medical'
    if 'moving' in title or 'relocation' in title:
        return 'moving'
    if 'vacation' in title:
        return 'vacation'
    if 'wedding' in title:
        return 'wedding'
    if 'renewable' in title or 'energy' in title:
        return 'renewable_energy'
    # 위 조건에 해당하지 않는 모든 경우는 'other'로 처리
    return 'other'

def preprocess_loan_data(accepted_path, rejected_path):
    """
    accepted.csv와 rejected.csv 파일을 불러와 대출 승인 예측 모델을 위한
    데이터 전처리 파이프라인을 실행 
    """
    try:
        # 데이터 타입 최적화로 메모리 사용량 줄이기
        accepted_cols_map = {
            'loan_amnt': np.float32, 'issue_d': str, 'purpose': str, 
            'dti': np.float32, 'zip_code': str, 'addr_state': str, 'emp_length': str
        }
        rejected_cols_map = {
            'Amount Requested': np.float32, 'Application Date': str, 'Loan Title': str,
            'Debt-To-Income Ratio': str, 'Zip Code': str, 'State': str, 'Employment Length': str
        }
        
        accepted_df = pd.read_csv(accepted_path, usecols=accepted_cols_map.keys(), dtype=accepted_cols_map)
        
   
        # [수정된 부분] ValueError 해결을 위해 skipinitialspace=True 추가
        # 컬럼 이름 주변의 공백을 무시하여 오류를 방지
        rejected_df = pd.read_csv(
            rejected_path, 
            usecols=rejected_cols_map.keys(), 
            dtype=rejected_cols_map,
            skipinitialspace=True 
        )

    except FileNotFoundError as e:
        print(f"오류: 파일 경로를 찾을 수 없습니다. {e}")
        return None
    except ValueError as e:
        print(f"오류: CSV 파일의 컬럼 이름이 코드와 일치하지 않을 수 있습니다. {e}")
        print("rejected.csv 파일의 실제 컬럼 이름과 rejected_cols_map 딕셔너리의 키가 동일한지 확인해주세요.")
        return None


    rejected_cols_rename = {
        'Amount Requested': 'loan_amnt', 'Application Date': 'issue_d', 'Loan Title': 'purpose',
        'Debt-To-Income Ratio': 'dti', 'Zip Code': 'zip_code', 'State': 'addr_state',
        'Employment Length': 'emp_length'
    }
    rejected_df.rename(columns=rejected_cols_rename, inplace=True)

    accepted_df['loan_status'] = 1
    rejected_df['loan_status'] = 0
    df = pd.concat([accepted_df, rejected_df], ignore_index=True)
    print(f"데이터 통합 완료. 총 데이터 수: {len(df)}개")

    # --- 컬럼별 전처리 ---
    print("'purpose' 컬럼 표준화 시작...")
    df['purpose'] = df['purpose'].apply(map_purpose)
    
    df['dti'] = df['dti'].astype(str).str.replace('%', '').astype(float).astype(np.float32)
    df['dti'].fillna(df['dti'].median(), inplace=True)

    df['emp_length'] = df['emp_length'].astype(str).str.replace('+', '', regex=False)
    df['emp_length'] = df['emp_length'].str.replace('< 1 year', '0 years', regex=False)
    df['emp_length'] = df['emp_length'].str.extract('(\d+)').astype(float).astype(np.float32)
    df['emp_length'].fillna(df['emp_length'].mode()[0], inplace=True)

    df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')
    df.dropna(subset=['issue_d'], inplace=True)
    df['issue_year'] = df['issue_d'].dt.year.astype(np.int16)
    df['issue_month'] = df['issue_d'].dt.month.astype(np.int8)
    df.drop('issue_d', axis=1, inplace=True)

    df['zip_code_prefix'] = df['zip_code'].astype(str).str[:3]
    df['zip_code_prefix'].fillna('000', inplace=True)
    le = LabelEncoder()
    df['zip_code_encoded'] = le.fit_transform(df['zip_code_prefix']).astype(np.int16)
    df.drop(['zip_code', 'zip_code_prefix'], axis=1, inplace=True)
    
    print("One-Hot Encoding 시작...")
    df = pd.get_dummies(df, columns=['purpose', 'addr_state'], dummy_na=False, dtype=np.int8)
    
    print("\n모든 전처리 과정이 완료되었습니다.")
    return df

if __name__ == '__main__':
    ACCEPTED_FILE_PATH = r'C:\Loan_Default_Prediction_MLOps\data\accepted_2007_to_2018Q4.csv'
    REJECTED_FILE_PATH = r'C:\Loan_Default_Prediction_MLOps\data\rejected_2007_to_2018Q4.csv'
    processed_df = preprocess_loan_data(ACCEPTED_FILE_PATH, REJECTED_FILE_PATH)

    if processed_df is not None:
        # 언더샘플링으로 승인/거절 비율 1:1로 맞추기

        print("\n--- 언더샘플링으로 클래스 비율 조정 시작 ---")
        
        # 원본 데이터의 클래스 비율 확인
        print("조정 전 클래스 비율:")
        print(processed_df['loan_status'].value_counts())
        
        # 0 (거절)과 1 (승인)으로 데이터 분리
        df_majority = processed_df[processed_df['loan_status'] == 0]
        df_minority = processed_df[processed_df['loan_status'] == 1]
        
        # 다수 클래스(거절)를 소수 클래스(승인)의 수만큼 무작위로 샘플링
        df_majority_downsampled = df_majority.sample(
            n=len(df_minority), 
            random_state=42 # 재현 가능성을 위해 시드 고정
        )
        
        # 샘플링된 다수 클래스와 소수 클래스 데이터를 다시 합치기
        df_balanced = pd.concat([df_majority_downsampled, df_minority])
        
        # 데이터를 무작위로 섞어주어 순서에 따른 편향 방지
        df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
        
        print("\n조정 후 클래스 비율:")
        print(df_balanced['loan_status'].value_counts())
        print(f"조정 후 전체 데이터 수: {len(df_balanced)}개")

        # 균형 잡힌 데이터 파일로 저장하기

        print("\n--- 균형 잡힌 데이터 파일로 저장 중... ---")
        output_filename = 'processed_loan_data_balanced.csv'
        
        df_balanced.to_csv(output_filename, index=False)
        
        print(f"'{output_filename}' 파일로 저장이 완료되었습니다.")



<>:99: SyntaxWarning: invalid escape sequence '\d'
<>:99: SyntaxWarning: invalid escape sequence '\d'
C:\Users\yoodongseok\AppData\Local\Temp\ipykernel_6556\3391973110.py:99: SyntaxWarning: invalid escape sequence '\d'
  df['emp_length'] = df['emp_length'].str.extract('(\d+)').astype(float).astype(np.float32)


데이터 통합 완료. 총 데이터 수: 29909442개
'purpose' 컬럼 표준화 시작...


C:\Users\yoodongseok\AppData\Local\Temp\ipykernel_6556\3391973110.py:95: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['dti'].fillna(df['dti'].median(), inplace=True)
C:\Users\yoodongseok\AppData\Local\Temp\ipykernel_6556\3391973110.py:100: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

One-Hot Encoding 시작...

모든 전처리 과정이 완료되었습니다.

--- 언더샘플링으로 클래스 비율 조정 시작 ---
조정 전 클래스 비율:
loan_status
0    27648741
1     2260668
Name: count, dtype: int64

조정 후 클래스 비율:
loan_status
1    2260668
0    2260668
Name: count, dtype: int64
조정 후 전체 데이터 수: 4521336개

--- 균형 잡힌 데이터 파일로 저장 중... ---
'processed_loan_data_balanced.csv' 파일로 저장이 완료되었습니다.


In [12]:
df = pd.read_csv(r"C:\Loan_Default_Prediction_MLOps\data\processed_loan_data_balanced.csv")
df.shape

(4521336, 70)